Notebook responsável para realizar transformações no dataframe de órgãos. Ele está sendo usado apenas para testes, código completos serão enviados para o arquivo de transform.py

Carregamento do arquivo bruto de contratos 👇

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from IPython.display import display
from datetime import datetime

# Configura o caminho para os arquivos brutos
raw_data_dir = Path('..') / 'data' / 'raw'

# Localiza o arquivo de órgãos mais recente (pelo formato de data no nome do arquivo)
orgaos_files = list(raw_data_dir.glob('orgao_*.csv'))

if orgaos_files:
    # Pega o arquivo mais recente baseado na data de modificação
    latest_orgaos_file = max(orgaos_files, key=lambda x: x.stat().st_mtime)

    # Verificação (para debug)
    print(f'Arquivo de órgãos encontrado: {latest_orgaos_file}')
    print(f'O arquivo existe? {latest_orgaos_file.exists()}')

    # Carrega o Dataframe
    df = pd.read_csv(latest_orgaos_file, encoding='utf-8', low_memory=False)

    # Visualiza dimensões e primeiras linhas
    print(f'\nDataframe carregado: {df.shape[0]} linhas, {df.shape[1]} colunas')
    print(f'Primeira 5 colunas: {', '.join(df.columns[:5])}')

else:
    print('ERRO: Nenhum arquivo de contratos encontrado em data/raw')
    print(f'Diretório verificado: {raw_data_dir.resolver()}')
    print(f'O diretório existe? {raw_data_dir.exists()}')
    print('Arquivos no diretório:', list(raw_data_dir.glob('*')))

Arquivo de órgãos encontrado: ..\data\raw\orgao_2025-08-03.csv
O arquivo existe? True

Dataframe carregado: 10017 linhas, 17 colunas
Primeira 5 colunas: codigoOrgao, nomeOrgao, nomeMnemonicoOrgao, cnpjCpfOrgao, codigoOrgaoVinculado


### ANÁLISE GERAL DO DATAFRAME

In [3]:
df.head(10)

,codigoOrgao,nomeOrgao,nomeMnemonicoOrgao,cnpjCpfOrgao,codigoOrgaoVinculado,cnpjCpfOrgaoVinculado,nomeOrgaoVinculado,codigoOrgaoSuperior,cnpjCpfOrgaoSuperior,nomeOrgaoSuperior,codigoTipoAdministracao,nomeTipoAdministracao,poder,esfera,usoSisg,statusOrgao,dataHoraMovimento
0,96845,PREFEITURA MUNICIPAL DE VERANOPOLIS - RS,PM VERANOPOLIS - RS,98671597000109,96320.0,8.793468e+13,ERS-ESTADO DO RIO GRANDE DO SUL,96000.0,0.000000e+00,REGIAO SUL,12,ADMINISTRACAO DIRETA MUNICIPAL,E,M,True,True,2024-01-14T13:57:00
1,96829,SETREM - SOC. EDUC. TRÊS DE MAIO,SOC E.TRES DE MAIO,98039852000600,96828.0,8.761280e+13,PREFEITURA MUNICIPAL DE TRÊS DE MAIO - RS,96320.0,8.793468e+13,ERS-ESTADO DO RIO GRANDE DO SUL,15,EMPRESA PRIVADA,E,M,True,True,2024-04-01T15:42:00
2,96817,PREFEITURA MUNICIPAL DE TAQUARA - RS,PM DE TAQUARA - RS,97761407000173,96320.0,8.793468e+13,ERS-ESTADO DO RIO GRANDE DO SUL,96000.0,0.000000e+00,REGIAO SUL,12,ADMINISTRACAO DIRETA MUNICIPAL,E,M,True,True,2024-01-14T13:57:00
3,87031,FUNDO MUNICIPAL DE SAUDE ARARI,FMS/ARARI,97552407000163,NaN,NaN,NaN,NaN,NaN,NaN,7,FUNDOS,E,M,True,True,2024-08-03T13:57:00
4,87643,FUNDO MUNICIPAL DE SAUDE JAÍBA,FMS/JAÍBA,97552158000106,NaN,NaN,NaN,NaN,NaN,NaN,7,FUNDOS,E,M,True,True,2024-08-11T13:57:00
5,88252,FUNDO M. S. SANTA CRUZ DA CONCEIÇÃO,FMS,97552000000136,NaN,NaN,NaN,NaN,NaN,NaN,7,FUNDOS,E,M,True,True,2024-08-24T13:57:00
6,87455,FUNDO MUNICIPAL DE SAUDE TASSO FRAGOSO,FMS/TASSO FRAGOSO,97551531000104,NaN,NaN,NaN,NaN,NaN,NaN,7,FUNDOS,E,M,True,True,2024-08-10T13:57:00
7,95161,EMG - CONSORCIO INTER.ALIANÇA PARA A SAUDE,CIAS - MG,97550393000149,95120.0,1.871562e+13,EMG-ESTADO DE MINAS GERAIS,95000.0,0.000000e+00,REGIAO SUDESTE,5,EMPRESA PUBLICA COMERCIAL E FINANCEIRA,E,E,True,True,2023-09-03T11:44:00
8,87550,FUNDO MUNICIPAL DE SAUDE GOIABEIRA,FMS/GOIABEIRA,97550158000177,NaN,NaN,NaN,NaN,NaN,NaN,7,FUNDOS,E,M,True,True,2024-08-10T13:57:00
9,87424,FUNDO MUNICIPAL DE SAUDE BACURI,FMS/BACURI,97546561000122,NaN,NaN,NaN,NaN,NaN,NaN,7,FUNDOS,E,M,True,True,2024-08-10T13:57:00


Verificação de Nulos por quantidade e porcentagem 👇

In [2]:
# Calcula contagem e percentual de nulos em cada coluna
nulos = df.isnull().sum()
percentual = (df.isnull().sum() / len(df) * 100).round(2)

# Cria um Dataframe simples apenas com colunas que têm pelo menos um valor nulo
colunas_com_nulos = pd.DataFrame({
    'Valores Nulos': nulos[nulos >0],
    'Percentual (%)': percentual[nulos > 0]
}).sort_values('Percentual (%)', ascending=False)

# Exibe o resultado
colunas_com_nulos

,Valores Nulos,Percentual (%)
codigoOrgaoSuperior,1415,14.13
cnpjCpfOrgaoSuperior,1415,14.13
nomeOrgaoSuperior,1415,14.13
codigoOrgaoVinculado,1276,12.74
cnpjCpfOrgaoVinculado,1276,12.74
nomeOrgaoVinculado,1276,12.74
poder,480,4.79
esfera,41,0.41
nomeMnemonicoOrgao,1,0.01
nomeTipoAdministracao,1,0.01


### ANÁLISE DAS COLUNAS NULAS